<a href="https://colab.research.google.com/github/iued-uni-heidelberg/corpusdev/blob/main/part2_word_embeddings_v05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word Vectors

Presentations [slides](intro-word-vectors-DAAD-2021-v22.pdf); Original [slides](word-vectors.pdf)

Word vectors (also known as 'word embeddings') are one of the most popular kinds of AI models. They are extremely useful in many domains. In essence, a word vector is a set of numbers that attempt to capture the meaning of a word. In typical implementations, each word is represented by a set of 200-300 numbers. In linear algebra, a one-dimensional array of numbers is known as a 'vector', hence these sets of numbers representing words' meanings are known as 'word vectors'.

Using neural networks, we can expose the computer to a large amount of text, and allow it to learn an appropriate set of numbers for each word it encounters. In this notebook, we will learn about the most famous of all word vector algorithms, `word2vec`, which was first described by Tomas Mikolov and his team in 2013:

* Tomas Mikolov, Ilya Sutskever, and others, ‘Distributed Representations of Words and Phrases and Their Compositionality’, in Advances in Neural Information Processing Systems 26, ed. by C. J. C. Burges and others (Curran Associates, Inc., 2013), pp. 3111–19 <http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf>
* Tomas Mikolov, Kai Chen, and others, ‘Efficient Estimation of Word Representations in Vector Space’, ArXiv:1301.3781 Cs, 2013 <http://arxiv.org/abs/1301.3781>.

In fact, `word2vec` is not a single algorithm, but rather a family of similar algorithms. In this session we will consider just the most famous `word2vec` algorithm, namely the `skip-gram model` trained using `negative sampling`.

## Applications of Word Vectors

Word vectors allow the computer to 'understand' language far more effectively. Rather than seeing each word as simply an arbitrarily different object, a computer using word vectors can analyse each word as a point in 200- or 300-dimenstional space. Words that are similar in meaning will have similar word vectors. And as we will see, the spaces between the word vectors are also significant: the words are arranged in patterns that represent their relationships to one another.

Accordingly, most AI systems that process language now include a word vector layer as part of their architecure. When the system encounters some text (e.g. when you speak to Siri or Alexa), your words are converted into word vectors, *and then* the computer examines what the text says and determines how it should respond.

In the Humanities, word vectors have become a popular modelling tool, because they allow researchers to perform sophisticated analysis on large corpora of text. Some examples include:

* [The Women Writers Vector Toolkit](https://wwp.northeastern.edu/lab/wwvt/index.html)
* William L. Hamilton, Jure Leskovec, and Dan Jurafsky, ‘Diachronic Word Embeddings Reveal Statistical Laws of Semantic Change’, ArXiv:1605.09096 [Cs], 2018 <http://arxiv.org/abs/1605.09096>.
* Ryan Heuser, 'Semantic Networks' <https://ryanheuser.org/word-vectors-4/>

## Training a `word2vec` model in Gensim

It is very easy to train a `word2vec` model in Gensim, which includes Mikolov's original `word2vec` code in its codebase.

In [1]:
from gensim.models import Word2Vec # The word2vec model class
import gensim.downloader as api # Allows us to download some free training data


[==================================================] 100.0% 31.6/31.6MB downloaded


In [ ]:
corpus = api.load('text8')
# api.info()

text8 is first 100MB from English wikipedia, 17 million words

The file is downloadable from
http://mattmahoney.net/dc/text8.zip


In [ ]:
# Examine the corpus to see what is there
api.info("text8")

In [ ]:
type(corpus)

In [4]:
data = [d for d in corpus]

In [5]:
len(data)

1701

In [6]:
len(data[0])

10000

In [ ]:
print(data[0][:100])

In [ ]:
print(data[1][:100])

### Step 1: Set hyperparameters and instantiate model

In [7]:
my_vector_size = 100 # Dimensionality of the word vectors
window = 5 # How many words either side? (5 = 5 context words either side, i.e. 10 context words in total)
use_skip_gram = 1 # If you set this to 0, then it will create a 'continuous bag of words' model instead
use_softmax = 0 # If you set this to 1, then hierarchical softmax will be used instead of negative sampling
negative_samples = 5 # How many incorrect answers to generate per correct answer when negative sampling

model = Word2Vec(
    vector_size=my_vector_size,
    window=window,
    sg=use_skip_gram,
    hs=use_softmax,
    negative=negative_samples
)

### Step 2: Fit model to corpus

In [8]:
# build a model
model.build_vocab(corpus)

In [ ]:
# Train the model on the corpus
# model.train(sentences=corpus, epochs=5, total_examples=model.corpus_count)
model.train(corpus, epochs=5, total_examples=model.corpus_count)

(62528051, 85026035)

In [ ]:
model.save("word2vec.model")

In [ ]:
!rm word2vec.model

In [ ]:
!wget https://heibox.uni-heidelberg.de/f/b7625cd746534aacbefe/?dl=1
!mv index.html?dl=1 word2vec.model

In [10]:
model = Word2Vec.load("word2vec.model")

### Step 3: Extract word vectors from model

The fully trained model includes all of the weights used to predict the context words for each input word. If you are not planning on training the model further, these weights can be discarded, and you can just keep the weights for the word vectors.

In [11]:
word_vectors = model.wv
# del model # Delete the whole model to free up the computer's RAM

In [12]:
word_vectors.save("word2vec.wordvectors")

In [ ]:
!rm word2vec.wordvectors

In [ ]:
!wget https://heibox.uni-heidelberg.de/f/e7725b1afab9456c8e16/?dl=1
!mv index.html?dl=1 word2vec.wordvectors

In [13]:
from gensim.models import KeyedVectors


In [15]:
wv = KeyedVectors.load("word2vec.wordvectors", mmap='r')

In [16]:
vector = wv['computer']

In [ ]:
vector

### Step 4: Have a play with the model

There are several ways you can use word vectors. One of the most famous is to use them to compute analogies. The formula is:

<center><em>x</em> is to <em>small</em> as <em>biggest</em> is to <em>big</em></center>

$$x - vector('small') = vector('biggest') - vector('big')$$

$$\therefore x = vector('small') + vector('biggest') - vector('big')$$

In [18]:
word_vectors = model.wv
del model # Delete the whole model to free up the computer's RAM

In [ ]:
# See the word vector for a particular word
vector = word_vectors['banana']
print(vector)

In [ ]:
# See which words are closest to a given word in the vector space
similar_words = word_vectors.most_similar('university', topn=10)
print('\n'.join([str(tup) for tup in similar_words]))

In [ ]:
# See which words are closest to a given word in the vector space
similar_words = word_vectors.most_similar('red', topn=10)
print('\n'.join([str(tup) for tup in similar_words]))

In [ ]:
# See which words are closest to a given word in the vector space
similar_words = word_vectors.most_similar('brussels', topn=10)
print('\n'.join([str(tup) for tup in similar_words]))

In [ ]:
# See which words are closest to a given word in the vector space
similar_words = word_vectors.most_similar('germany', topn=10)
print('\n'.join([str(tup) for tup in similar_words]))

In [ ]:
# See which words are closest to a given word in the vector space
similar_words = word_vectors.most_similar('strange', topn=10)
print('\n'.join([str(tup) for tup in similar_words]))

In [ ]:
# Compute analogous words
# E.g. x is to queen as man is to king => x = v('queen') + v('man') - v('king')
analogous_words = word_vectors.most_similar(negative=['man'], positive=['king','woman'])
print('\n'.join([str(tup) for tup in analogous_words]))

In [ ]:
# Compute analogous words
# E.g. x is to queen as man is to king => x = v('queen') + v('man') - v('king')
analogous_words = word_vectors.most_similar(negative=['king'], positive=['queen','man'])
print('\n'.join([str(tup) for tup in analogous_words]))

In [ ]:
# Compute analogous words
# E.g. x is to queen as man is to king => x = v('queen') + v('man') - v('king')
analogous_words = word_vectors.most_similar(negative=['mother'], positive=['father','daughter'])
print('\n'.join([str(tup) for tup in analogous_words]))

In [ ]:
# Compute analogous words
# E.g. x is to queen as man is to king => x = v('queen') + v('man') - v('king')
analogous_words = word_vectors.most_similar(negative=['mother'], positive=['father','girl'])
print('\n'.join([str(tup) for tup in analogous_words]))

In [ ]:
# Compute analogous words
# x is to daughter as people is to person (plural + daughter)
analogous_words = word_vectors.most_similar(negative=['person'], positive=['people','daughter'])
print('\n'.join([str(tup) for tup in analogous_words]))

In [ ]:
analogous_words = word_vectors.most_similar(negative=['soft'], positive=['hard','softer'])
# analogous_words = word_vectors.most_similar(negative=['soft'], positive=['hard','softest'])
print('\n'.join([str(tup) for tup in analogous_words]))

Sentence similarity
"And break your heavy chains"

In [ ]:
from gensim.test.utils import lee_corpus_list
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.test.utils import datapath
import gensim.downloader as api

In [28]:
result = word_vectors.most_similar(positive=['woman', 'king'], negative=['man'])
most_similar_key, similarity = result[0]
print(f"{most_similar_key}: {similarity:.4f}")

queen: 0.6900


In [29]:
result = word_vectors.most_similar_cosmul(positive=['woman', 'king'], negative=['man'])
most_similar_key, similarity = result[0]
print(f"{most_similar_key}: {similarity:.4f}")

queen: 0.9126


In [30]:
print(word_vectors.doesnt_match("breakfast cereal dinner lunch".split()))

cereal


In [32]:
result = word_vectors.similar_by_word("dog")
most_similar_key, similarity = result[0]
print(f"{most_similar_key}: {similarity:.4f}")

hound: 0.7839


In [33]:
sentence_obama = 'Obama speaks to the media in Illinois'.lower().split()
sentence_president = 'The president greets the press in Chicago'.lower().split()
print(sentence_obama)
print(sentence_president)

['obama', 'speaks', 'to', 'the', 'media', 'in', 'illinois']
['the', 'president', 'greets', 'the', 'press', 'in', 'chicago']


In [ ]:
pip install POT

In [36]:
similarity = word_vectors.wmdistance(sentence_obama, sentence_president)

In [37]:
print(f"{similarity:.4f}")

0.6707


In [42]:
sentence_obama = 'And water with the tyrant blood the freedom you have gained'.lower().split()
sentence_president = 'resistance oppression'.lower().split()
similarity = word_vectors.wmdistance(sentence_obama, sentence_president)
print(sentence_obama)
print(sentence_president)
print(f"{similarity:.4f}")

['and', 'water', 'with', 'the', 'tyrant', 'blood', 'the', 'freedom', 'you', 'have', 'gained']
['resistance', 'oppression']
1.1306


In [44]:
# result = word_vectors.most_similar_cosmul(positive=['woman', 'king'], negative=['man'])
analogous_words = word_vectors.most_similar(positive='And water with the tyrant blood the freedom you have gained'.lower().split())
print('\n'.join([str(tup) for tup in analogous_words]))

('energizing', 0.829910933971405)
('uncompromisingly', 0.8271280527114868)
('transfigured', 0.8253328204154968)
('cynically', 0.8250075578689575)
('cutthroat', 0.8245701193809509)
('faking', 0.8226403594017029)
('complacency', 0.82034832239151)
('obviate', 0.8198023438453674)
('shortlived', 0.8193068504333496)
('unvisited', 0.8188250064849854)


In [48]:
# The Dnieper plunging shore My eyes could see my ears could hear The mighty river roar
# result = word_vectors.most_similar_cosmul(positive=['woman', 'king'], negative=['man'])
analogous_words = word_vectors.most_similar(positive='So that the fields the boundless steppes'.lower().split())
print('\n'.join([str(tup) for tup in analogous_words]))

('energizing', 0.8446081876754761)
('straightened', 0.833925187587738)
('plowed', 0.8328710794448853)
('ergosphere', 0.8307587504386902)
('unassailable', 0.8307085633277893)
('obviate', 0.8282104134559631)
('infinitesimally', 0.8248646855354309)
('equalized', 0.8247248530387878)
('assimilating', 0.8237602710723877)
('underpin', 0.8231127858161926)


In [50]:
analogous_words = word_vectors.most_similar(positive='The Dnieper plunging shore My eyes could see my ears could hear The mighty river roar'.lower().split())
print('\n'.join([str(tup) for tup in analogous_words]))

('yonder', 0.8880959153175354)
('forlorn', 0.8635091781616211)
('faking', 0.8602349758148193)
('rug', 0.859780490398407)
('crom', 0.8596680760383606)
('thro', 0.8581385016441345)
('blindfolded', 0.8574162125587463)
('rained', 0.8555653095245361)
('forbear', 0.8549303412437439)
('fortifying', 0.8526374101638794)


In [ ]:
'''
from ot import emd2
sentence_obama = 'Obama speaks to the media in Illinois'.lower().split()
sentence_president = 'The president greets the press in Chicago'.lower().split()

similarity = word_vectors.wmdistance(sentence_obama, sentence_president)
print(f"{similarity:.4f}")
'''

In [25]:
similarity = word_vectors.similarity('king', 'queen')
print(f"{similarity:.4f}")

0.7212


In [26]:
similarity = word_vectors.similarity('man', 'woman')
print(f"{similarity:.4f}")

0.7245


In [ ]:
# similarity = word_vectors.similarity('emotion', positive=['people','daughter'])
# print(f"{similarity:.4f}")

## Step 4A - optional. Using pre-trained models in Gensim

In many applications, you will simply want access to pre-trained word vectors (e.g. for plugging in to another model you are training). If you don't need the vectors to be tailored closely to your particular corpus, then you might like to use some pretrained models.

`word2vec` is not the only word embedding family of algorithms. Another, arguably even more powerful algorithm is the `FastText` algorithm, which Mikolov developed after moving to Facebook:

* Piotr Bojanowski and others, ‘Enriching Word Vectors with Subword Information’, ArXiv:1607.04606, 2017 <http://arxiv.org/abs/1607.04606>.

Instead of computing word vectors for each word, FastText splits each word into its constituent chunks. For example, 'cat' would be split into 'c', 'a', 't', 'ca', 'at' and 'cat', and 'burp' would be split into 'b', 'u', 'r', 'p', 'bu', 'ur', 'rp', 'bur', 'urp' and 'burp'. Then a vector is computer for each chunk that appears in the corpus. Each word is represented as the mean of all the chunks that make it up. FastText is able to learn very good word vectors because it can extract meaning from subword units, e.g. it can see that 'television', 'telegraph' and 'telepathy' all have 'tele' at the front, and can see that 'formality', 'criminality' and 'paucity' share subword units such as 'al' and 'ity'.

You can access many pretrained models using the Gensim downloader. Using the cells below, you can try out some of the different models available through Gensim. Along with `word2vec` and `FastText`, Gensim also supports `Glove` and `Doc2Vec` models.

**NB:** These trained models are very large, and will take a while to download. You may wish to download this notebook and execute the cells below on your own machine, in case Google kicks you out of the Colab environment.

In [ ]:
# See what models are on offer
print(list(api.info()['models'].keys()))

In [ ]:
# Optional! THIS WILL TAKE SOME TIME!!!
# 300-dimensional word vectors trained on a huge dataset from Google News
google_news_w2v = api.load('word2vec-google-news-300')

In [ ]:
# x is to Kenya as Canberra is to Australia
google_news_w2v.most_similar(negative=['australia'],positive=['kenya','canberra'], topn=10)

In [ ]:
# Optional! THIS WILL TAKE SOME TIME!!!
# Facebook's own FastText vectors, trained on Wikipedia
wikipedia_fasttext = api.load('fasttext-wiki-news-subwords-300')

In [ ]:
# x is to Wharton as London is to Dickens
wikipedia_fasttext.most_similar(negative=['dickens'],positive=['wharton','london'], topn=10)

In [ ]:
# try your own examples...

## Step 5. Training models in other languages (Armenian, Georgian, Ukrainian ...)

Now let's train a model on our own corpus. You can try to create Armenian / Ukrainian / Russian / German / French word vector model.

Armenian, Ukrainian, Russian plaintext wikipedias are available at:
https://lindat.mff.cuni.cz/repository/xmlui/handle/11234/1-2735#

We can use the same size ~15 Million Words, (200MB) of these corpora to make sure the model is built within a reasonable time; however, when you have more time, you can try a more complete corpus.

It is possible to download/unzip/upload the files manually, however, it is faster to automatically load them directly from Heidelberg server space https://heibox.uni-heidelberg.de/d/42a07c9e95774e099a81/  into your colab environment. To do this run the cells 5a or 5b, or 5c, or 5d, or 5e -- depending on which language you prefer -- to download the data. Then skip other language and continue with Step5 -- building the model for your preferred language from the data you downloaded.


In [ ]:
!rm /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt
!rm index.html?dl=1

rm: cannot remove '/usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt': No such file or directory
rm: cannot remove 'index.html?dl=1': No such file or directory


### Either: 5a Armenian

In [ ]:
!wget https://heibox.uni-heidelberg.de/f/ffe527ed3d1e4b4cb43a/?dl=1

In [ ]:
!cp index.html?dl=1 /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt

In [ ]:
!head --lines=15 /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt

### Or: 5b Ukrainian

In [ ]:
!wget https://heibox.uni-heidelberg.de/f/676bef3a6db8482e9665/?dl=1

In [ ]:
!cp index.html?dl=1 /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt

In [ ]:
!head --lines=15 /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt

### Or: 5c German

In [ ]:
!wget https://heibox.uni-heidelberg.de/f/ca0a085347c34563a6e6/?dl=1

In [ ]:
!cp index.html?dl=1 /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt

In [ ]:
!head --lines=15 /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt

### Or: 5d French

In [ ]:
!wget https://heibox.uni-heidelberg.de/f/5cdb485efd4046f2a457/?dl=1

In [ ]:
!cp index.html?dl=1 /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt

In [ ]:
!head --lines=15 /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt

### Or: 5f Georgian

In [ ]:
!wget https://heibox.uni-heidelberg.de/f/fa3509d869b949459f91/?dl=1

Lemmatized corpus

In [ ]:
!wget https://heibox.uni-heidelberg.de/f/b381c458ad5e4773a77d/?dl=1

In [ ]:
!cp index.html?dl=1 wiki_geo_lem.txt

In [ ]:
!head --lines=10 wiki_geo_lem.txt

In [ ]:
FIn = open("wiki_geo_lem.txt", 'r')
FOut = open('wiki_geo_l.txt', 'w')

i = 0
for SLine in FIn:
  i = i + 1
  SLine = SLine.strip() + ' '
  if i % 10000 == 0:
    SLine += '\n'

  FOut.write(SLine)

FOut.flush()
FOut.close()


In [ ]:
!head --lines=10 wiki_geo_l.txt

In [ ]:
!wc wiki_geo_lem.txt

 18054750  18054749 329202500 wiki_geo_lem.txt


In [ ]:
!cp index.html?dl=1 /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt

Alternatively, copy lemmatized corpus

In [ ]:
!cp wiki_geo_l.txt /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt

In [ ]:
!head --lines=15 /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt

### Stage6: Training own model

optional: to clean disk space, we remove the downloaded file

In [ ]:
# optional: to clean disk space, we remove the downloaded file
!rm index.html\?dl\=1

In [ ]:
from gensim.test.utils import datapath
from gensim import utils

class MyCorpus:
    """An iterator that yields sentences (lists of str)."""

    def __iter__(self):
        corpus_path = datapath('myOwnLangText8.txt')
        for line in open(corpus_path):
            # assume there's one document per line, tokens separated by whitespace
            yield utils.simple_preprocess(line)

In [ ]:
import gensim.models
corpusOwn = MyCorpus()

optional: examining what is in the corpus after standard preprocessing

In [ ]:
# Optional: Examining our corpus format
type(corpusOwn)

__main__.MyCorpus

In [ ]:
dataOwn = [d for d in corpusOwn]

In [ ]:
print(len(dataOwn))
print(dataOwn[0])
print(len(dataOwn[0]))
print(dataOwn[4])
print(len(dataOwn[4]))
print(dataOwn[5])
print(len(dataOwn[5]))

... Initialising global parameters for our modelL vector size, collocation window, skip-grams, negative sampling....

In [ ]:
from gensim.models import Word2Vec # The word2vec model class
import gensim.downloader as api # Allows us to download some free training data

In [ ]:
vector_size = 100 # Dimensionality of the word vectors
window = 5 # How many words either side? (5 = 5 context words either side, i.e. 10 context words in total)
use_skip_gram = 1 # If you set this to 0, then it will create a 'continuous bag of words' model instead
use_softmax = 0 # If you set this to 1, then hierarchical softmax will be used instead of negative sampling
negative_samples = 5 # How many incorrect answers to generate per correct answer when negative sampling

modelOwn = Word2Vec(
    size=vector_size,
    window=window,
    sg=use_skip_gram,
    hs=use_softmax,
    negative=negative_samples
)

... this cell may run for ~2 min or so...

In [ ]:
modelOwn.build_vocab(corpusOwn)

THIS MAY TAKE LONG!!! ... training the model may take 9 to 15 minutes... (just grab a cup of coffee or a sandwich while you are waiting... You may try chaning the number of epochs; if the number is lower the training is faster, but the quality may be lower...

In [ ]:
modelOwn.train(sentences=corpusOwn, epochs=5, total_examples=modelOwn.corpus_count)

(59378395, 70859245)

Now we copy word vectors and remove the model from memory (just to free up the resources...)

In [ ]:
word_vectors_own = modelOwn.wv

In [ ]:
del modelOwn

Now we can examine the output of our word embeddings

### German examples

In [ ]:
# DE examples
# See the word vector for a particular word
vector = word_vectors['banane']
print(vector)

In [ ]:
# See which words are closest to a given word in the vector space
similar_words = word_vectors.most_similar('universität', topn=10)
print('\n'.join([str(tup) for tup in similar_words]))

In [ ]:
# See which words are closest to a given word in the vector space
similar_words = word_vectors.most_similar('rot', topn=10)
print('\n'.join([str(tup) for tup in similar_words]))

In [ ]:
# See which words are closest to a given word in the vector space
similar_words = word_vectors.most_similar('brüssel', topn=10)
print('\n'.join([str(tup) for tup in similar_words]))

In [ ]:
# See which words are closest to a given word in the vector space
similar_words = word_vectors.most_similar('deutschland', topn=10)
print('\n'.join([str(tup) for tup in similar_words]))

In [ ]:
# See which words are closest to a given word in the vector space
similar_words = word_vectors.most_similar('komisch', topn=10)
print('\n'.join([str(tup) for tup in similar_words]))

In [ ]:
# Compute analogous words
# E.g. x is to queen as man is to king => x = v('queen') + v('man') - v('king')
analogous_words = word_vectors.most_similar(negative=['mann'], positive=['könig','frau'])
print('\n'.join([str(tup) for tup in analogous_words]))

In [ ]:
# Compute analogous words
# E.g. x is to queen as man is to king => x = v('queen') + v('man') - v('king')
analogous_words = word_vectors.most_similar(negative=['könig'], positive=['königin','mann'])
print('\n'.join([str(tup) for tup in analogous_words]))

In [ ]:
# Compute analogous words
# E.g. x is to queen as man is to king => x = v('queen') + v('man') - v('king')
analogous_words = word_vectors.most_similar(negative=['mutter'], positive=['vater','tochter'])
print('\n'.join([str(tup) for tup in analogous_words]))

In [ ]:
# Compute analogous words
# E.g. x is to queen as man is to king => x = v('queen') + v('man') - v('king')
analogous_words = word_vectors.most_similar(negative=['mutter'], positive=['vater','mädchen'])
print('\n'.join([str(tup) for tup in analogous_words]))

In [ ]:
# Compute analogous words
# x is to daughter as people is to person (plural + daughter)
analogous_words = word_vectors.most_similar(negative=['person'], positive=['menschen','tochter'])
print('\n'.join([str(tup) for tup in analogous_words]))

In [ ]:
analogous_words = word_vectors.most_similar(negative=['weich'], positive=['hart','weichste'])
print('\n'.join([str(tup) for tup in analogous_words]))

### Georgian examples

In [ ]:
# See the word vector for a particular word -- 'world'
vector = word_vectors_own['სამყარო']
print(vector)

In [ ]:
# See which words are closest to a given word in the vector space = 'world'
similar_words = word_vectors_own.most_similar('არსება', topn=30)
print('\n'.join([str(tup) for tup in similar_words]))

In [ ]:
# blue
similar_words = word_vectors_own.most_similar('ლურჯი', topn=30)
print('\n'.join([str(tup) for tup in similar_words]))

In [ ]:
# france
similar_words = word_vectors_own.most_similar('საფრანგეთი', topn=10)
print('\n'.join([str(tup) for tup in similar_words]))

In [ ]:
# x is to king as woman is to man
analogous_words = word_vectors_own.most_similar(negative=['კაცი'], positive=['მეფე','ქალი'])
print('\n'.join([str(tup) for tup in analogous_words]))

In [ ]:
# x is to king as woman is to man
# x = small + biggest - big
# 𝑥−𝑣𝑒𝑐𝑡𝑜𝑟(′𝑠𝑚𝑎𝑙𝑙′)=𝑣𝑒𝑐𝑡𝑜𝑟(′𝑏𝑖𝑔𝑔𝑒𝑠𝑡′)−𝑣𝑒𝑐𝑡𝑜𝑟(′𝑏𝑖𝑔′)
# ∴ 𝑥=𝑣𝑒𝑐𝑡𝑜𝑟(′𝑠𝑚𝑎𝑙𝑙′)+𝑣𝑒𝑐𝑡𝑜𝑟(′𝑏𝑖𝑔𝑔𝑒𝑠𝑡′)−𝑣𝑒𝑐𝑡𝑜𝑟(′𝑏𝑖𝑔′)
# ∴ 𝑥=𝑣𝑒𝑐𝑡𝑜𝑟(′փոքր′)+𝑣𝑒𝑐𝑡𝑜𝑟(′ամենամեծը′)−𝑣𝑒𝑐𝑡𝑜𝑟(′մեծ′)
analogous_words = word_vectors_own.most_similar(negative=['პირი'], positive=['ხალხი','ქალიშვილი'])
print('\n'.join([str(tup) for tup in analogous_words]))

In [ ]:
# try your own examples... (also -- morphology?)
# # x is to daughter as people is to person (plural + daughter)
# analogous_words = word_vectors.most_similar(negative=['person'], positive=['people','daughter'])

### Armenian examples

In [ ]:
# See the word vector for a particular word -- 'world'
vector = word_vectors_own['աշխարհը']
print(vector)

In [ ]:
# See which words are closest to a given word in the vector space = 'world'
similar_words = word_vectors_own.most_similar('աշխարհը', topn=10)
print('\n'.join([str(tup) for tup in similar_words]))

In [ ]:
# blue
similar_words = word_vectors_own.most_similar('կապույտ', topn=10)
print('\n'.join([str(tup) for tup in similar_words]))

In [ ]:
# france
similar_words = word_vectors_own.most_similar('ֆրանսիա', topn=10)
print('\n'.join([str(tup) for tup in similar_words]))

In [ ]:
# x is to king as woman is to man
analogous_words = word_vectors_own.most_similar(negative=['մարդ'], positive=['թագավոր','կին'])
print('\n'.join([str(tup) for tup in analogous_words]))

In [ ]:
# x is to king as woman is to man
# x = small + biggest - big
# 𝑥−𝑣𝑒𝑐𝑡𝑜𝑟(′𝑠𝑚𝑎𝑙𝑙′)=𝑣𝑒𝑐𝑡𝑜𝑟(′𝑏𝑖𝑔𝑔𝑒𝑠𝑡′)−𝑣𝑒𝑐𝑡𝑜𝑟(′𝑏𝑖𝑔′)
# ∴ 𝑥=𝑣𝑒𝑐𝑡𝑜𝑟(′𝑠𝑚𝑎𝑙𝑙′)+𝑣𝑒𝑐𝑡𝑜𝑟(′𝑏𝑖𝑔𝑔𝑒𝑠𝑡′)−𝑣𝑒𝑐𝑡𝑜𝑟(′𝑏𝑖𝑔′)
# ∴ 𝑥=𝑣𝑒𝑐𝑡𝑜𝑟(′փոքր′)+𝑣𝑒𝑐𝑡𝑜𝑟(′ամենամեծը′)−𝑣𝑒𝑐𝑡𝑜𝑟(′մեծ′)
analogous_words = word_vectors_own.most_similar(negative=['մեծ'], positive=['ամենամեծը','փոքր'])
print('\n'.join([str(tup) for tup in analogous_words]))

In [ ]:
# try your own examples... (also -- morphology?)
# # x is to daughter as people is to person (plural + daughter)
# analogous_words = word_vectors.most_similar(negative=['person'], positive=['people','daughter'])